# Prolog to Query Databases

Prolog is a declarative programming language that is particularly useful for solving problems related to artificial intelligence and natural language processing. Prolog stands for "Programming in Logic," which indicates that Prolog programs are based on logical rules rather than procedural rules. This makes Prolog particularly well-suited to problems that involve searching through large amounts of data or reasoning about complex systems.



## Features of Prolog

Prolog is a unique programming language that offers several features that set it apart from other languages:

- **Declarative syntax**: Prolog programs are written using declarative syntax, which means that the program describes the problem to be solved rather than providing a step-by-step procedure for solving it.
- **Logical programming**: Prolog is based on logical programming, which means that programs are based on a set of logical rules that describe the problem domain.
- **Backtracking**: Prolog uses a backtracking search algorithm to find solutions to problems. This allows the programmer to explore multiple possible solutions to a problem.
- **Built-in support for data types**: Prolog includes built-in support for several data types, including numbers, strings, and lists.
- **Inference engine**: Prolog includes an inference engine that can be used to reason about logical statements and make deductions based on them.



## Benefits and drawbacks of Prolog

Prolog has several benefits and drawbacks that are important to consider when choosing a database technology. 

#### Some of the benefits of Prolog include:

- ****Easy to learn****: Prolog has a simple and easy-to-understand syntax, which makes it easy for programmers to learn.
- ****Efficient search****: Prolog's backtracking search algorithm is particularly efficient for exploring multiple solutions in a database.
- ****Natural language processing****: Prolog is particularly well-suited for problems related to natural language processing, as its logical rules can be used to reason about language structures.

#### Some of the drawbacks of Prolog include:

- **Limited scalability**: Prolog is not well-suited for very large databases, as its search algorithm can become slow and inefficient.
- **Steep learning curve**: Although Prolog's syntax is simple, its logical programming paradigm can be difficult for some programmers to understand.
- **Limited industry support**: Prolog is not widely used in industry, so finding support and resources can be challenging.



## Real world applications of Prolog 

Prolog has been used in several real-world applications, including:

- **Expert systems**: Prolog's ability to reason about logical statements has made it a popular choice for developing expert systems that can provide advice and recommendations based on input data.
- **Natural language processing**: Prolog has been used to develop several natural language processing systems, including chatbots and question-answering systems.
- **Healthcare**: Prolog has been used in several healthcare applications, including diagnosing diseases and creating treatment plans based on patient data.


# Tutorial
This tutorial will be a good foundation on how to use Prolog if you're a beginner on the subject and if you're already familiar with Prolog, you can use it to have a little reminder on the basics.

## SWI-Prolog

First, let's make sure that you have [SWI-Prolog](https://www.swi-prolog.org) installed on your machine by running the following commands. This will download the SWI-Prolog, which is a free and portable Prolog environment that is easy to use and compliant with textbooks, using the PPA (Ubuntu Personnal Package Archive). 

    apt-get install software-properties-common
    apt-add-repository ppa:swi-prolog/stable
    apt-get update
    apt-get install swi-prolog

If you're a Mac user, you can install SWI-Prolog using [Homebrew](https://brew.sh/) by running the following command (be sure you have Homebrew installed on your machine), if not you can install it by running this simple command : 

    /bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"

The simply run the next command to install SWI-Prolog :

    /bin/bash brew install swi-prolog

If you're not a Ubuntu nor Mac user, we strongly advise you to use the following Notebook in [Google Colab](https://colab.research.google.com) as the build of SWI-Prolog on Windows is not as easy as on Linux or Mac.

![](https://www.swi-prolog.org/icons/swipl.png)

## Library

In this tutorial, we'll use a package to allow us to query databases in Python by using the `SWI-Prolog` environment. This package is named `PySWIP` and can be installed with the following command (building from the Github repository as the package available on [PyPI](https://pypi.org/project/pyswip/) is not up to date and creates errors).

    pip install git+https://github.com/yuce/pyswip@master#egg=pyswip

Let's now jump into what's important, the code !

## Prolog in PySWIP

With the `PySWIP` package installed, we can create a `Prolog` object that will allow us to query with Prolog and use the `assertz()` function to create facts about the instances in our database. 

In [ ]:
from pyswip import *

prolog = Prolog()

prolog.assertz("father(john, jim)")
prolog.assertz("mother(jane, jim)")

prolog.assertz("father(john, jenny)")
prolog.assertz("mother(jane, jenny)")

prolog.assertz("father(john, jack)")
prolog.assertz("father(mary, jack)")

prolog.assertz("father(rob, andrew)")
prolog.assertz("mother(mary, andrew)")

prolog.assertz("father(bob, billy)")
prolog.assertz("mother(sue, billy)")

# Prolog basics

In Prolog, everything is based on facts and rules. Facts are specified in the format as seen in the cell above and represent informations known about our dataset. We can deduce new relations between instances using the rules (by doing queries).

A query is represented by a fact followed by the parameters between parantheses followed by `:-` the part of the query where the rule is formulated.

- `,` represents the logical AND operator

- Writing two defintions for the same fact will be interpreted as a logical OR operator

- `.` delimits the end of a query

- `\+` represents the logical NOT operator

- `[{}]` represents True

- `[]` represents False


Simples queries over the facts will simply return True or False, but we can go further by doing variable queries.

When using pyswip we can use the `query()` method to query the database and get the results. The results are returned as a list of dictionaries, where each dictionary represents a result and the keys are the variables used in the query. But we still need to explicitly use the `list()` function to get the results.

We can create a small function that can simply check if a query returns True or False and can start checking some facts about the relations we've just created :

In [ ]:
# How to do a simple query

def check_query(query):
    query = list(query)
    return len(query) != 0

print(check_query(prolog.query("father(john, jim)")))
print(check_query(prolog.query("mother(sue, jim)")))

In [ ]:
# How to do a variable query

def print_query(query, field):
    query = list(query)
    for result in query:
        print(result[field])

jims_father = prolog.query("father(X, jim)")

print("Jim's father :")
print_query(jims_father, 'X')

In [ ]:
rule_parent1 = "parent(X, Y) :- father(X, Y)"
rule_parent2 = "parent(X, Y) :- mother(X, Y)"

prolog.assertz(rule_parent1)
prolog.assertz(rule_parent2)

We can now create our own rule to query this small database to look for siblings :

In [ ]:
# Rule :
rule_sibling = "sibling(X, Y) :- father(F, X), father(F, Y), mother(M, X), mother(M, Y), X \= Y"

# Assert the rule
prolog.assertz(rule_sibling)

# Check the rule
jims_siblings = prolog.query("sibling(X, jim)")

print("Jim's siblings :")
print_query(jims_siblings, 'X')

Can you now come up with a rule to check for stepsiblings ?

In [ ]:
# Rules :
rule_step_sibling = "step_sibling(X, Y) :- parent(P, X), parent(P, Y), parent(Q, X), parent(R, Y), P \= Q, P \= R, Q \= R"

# Assertion
prolog.assertz(rule_step_sibling)

# Check the rules
jacks_step_siblings = prolog.query("step_sibling(X, jack)")
print("Jack's step siblings :")
print_query(jacks_step_siblings, 'X')

# Cut

The cut operator `!` is used to stop the search for a solution when a solution is found. It is used to avoid backtracking and to improve the efficiency of the search. It is used in the following way :

    query :- condition1, !, condition2.

This will only succeed if `condition1` is true and `condition2` is true. If `condition1` is true and `condition2` is false, the query will fail. If `condition1` is false, the query will fail.

This is a simple example :

In [ ]:
cut_prolog = Prolog()
cut_prolog.assertz("p(a)")
cut_prolog.assertz("p(b)")
cut_prolog.assertz("p(c)")
 
print_query(cut_prolog.query("p(X)"), 'X')

In the following code, we can see the query will stop immediatly when it finds a cut operator and will not backtrack to find other solutions.

In [ ]:
cut_prolog.assertz("p1(a)")
cut_prolog.assertz("p1(b):-!")
cut_prolog.assertz("p1(c)")
 
print_query(cut_prolog.query("p1(X)"), 'X')

The following code is a logic extension of what we just see

In [ ]:
list(cut_prolog.query( "p1(X),p1(Y)"))

Just bellow, the query is a bit more complex.
If we reuse the expression bellow: 

    query :- condition1, !, condition2.

Here the fist condition is a conditionnal operator but with the cut operator in `p(b))` we can see that the query will stop immediatly when it finds a cut operator and immediatly return the result wich also a conditionnal operator and has only 2 answers because of the cut operator in the declaration of `p(b)`.

In [ ]:
list(cut_prolog.query( "p1(X),!,p1(Y)"))

Let see a more usefull example :

In [ ]:
sign = Prolog()
sign.assertz("sign(X, positive):- X > 0")
sign.assertz("sign(X, negative):- X < 0")
sign.assertz("sign(X, zero):- X == 0")

print_query(sign.query("sign(1, X)"), 'X')
print_query(sign.query("sign(-1, X)"), 'X')

In this example, prolog will test all the 3 conditions and will return all the possible solutions. But we know that there is only one solution, so we can use the cut operator to stop the search for a solution when a solution is found and improve the efficiency of the search.

In [ ]:
sign.assertz("sign1(X, positive):- X > 0, !")
sign.assertz("sign1(X, negative):- X < 0, !")
sign.assertz("sign1(X, zero):- X == 0")

print_query(sign.query("sign1(1, X)"), 'X')
print_query(sign.query("sign1(-1, X)"), 'X')

Now, let's have a little exercise to test your knowledge on the cut operator. Modify the following code to improve its efficiency.

In [ ]:
#Add 2 cut operators to make this automatic comment on a student's grade more efficient

note = Prolog()
note.assertz("note(X, Failed) :- X < 10")
note.assertz("note(X, Sufficient) :- X =< 10")
note.assertz("note(X, Great) :- X > 10")


The code above as the same effect but stop the search for a solution when a solution is found. This is what we call a **green cut** because the answer is the same as the query without cut operator.

## Negation as Failure

Negation as failure (NAF) is a technique that allows us to query the database for a fact that is not present in the database. This is done by using the `\+` operator. We can illustrate this by searching for unique children in our database :

In [ ]:
# Rules :
rule_unique_child = "unique_child(X) :- \+ sibling(X, _), \+ step_sibling(X, _)"

# Assertion
prolog.assertz(rule_unique_child)

juliette_unique_child = prolog.query("unique_child(juliette)")
check_query(juliette_unique_child)

This operator can be defined using the cut operator as seen before and a new keyword `fail` that will always fail the query.
The implementation of a `neg` operator wich is the same as `\+` operator is as follows :

    neg(Goal) :- Goal, !, fail.
    neg(_) :- true.

This operator is a bit confusing because it will not tell us that a fact is False but will tell us that a fact is not present in the database. This is what we call the closed world assumption.

Now, a little exercise to test your knowledge on the negation as failure operator. In the following code, there is a list of students and a list a graduated students. Modify the code to get the list of students that are not graduated.

In [ ]:
students = Prolog()
students.assertz("student(s123456)")
students.assertz("student(s123457)")
students.assertz("student(s123458)")
students.assertz("student(s123459)")
students.assertz("student(s123460)")

students.assertz("grade(s123456)")
students.assertz("grade(s123457)")


## Functors

In Prolog, functors represent the name-arity pair that is used to differentiate between different predicates.

It is essentially a compound term that consists of a name and an arity, separated by a forward slash ("/"). The name represents the functor's identifier, while the arity specifies the number of arguments the functor takes. So in the previous examples, the functors are `father/2`, `mother/2`, `parent/2` and `sibling/2`.

Functors can be used to create new predicates and evaluates expression directly at runtime which is useful for many tasks such as constraint solving, data manipulation, symbolic computation, and more.

In summary, functors are an important concept in Prolog that allows programmers to create and manipulate compound terms and predicates in a flexible and dynamic way.

Let us for example define a functor `book/4` to represent a book with it's title, author, genre and year of publication. We can declare some books and more complex queries.

In [ ]:
books = Prolog()

books.assertz("book('The Lord of the Rings', tolkien, fantasy, 1954)")
books.assertz("book('The Hobbit', tolkien, fantasy, 1937)")
books.assertz("book('To Kill a Mockingbird', lee, fiction, 1960)")
books.assertz("book('The Great Gatsby', fitzgerald, fiction, 1925)")
books.assertz("book('Les Miserables', hugo, fiction, 1862)")
books.assertz("book('Orient Express', christie, mystery, 1934)")
books.assertz("book('The Da Vinci Code', brown, mystery, 2003)")

# Let's now do some queries on the books :

# Query 1 : Find all the books written by Tolkien
books_tolkien = books.query("book(X, tolkien, Y, Z)")
print("\nBooks written by Tolkien :")
print_query(books_tolkien, 'X')

# We can also use the following syntax :
# Query 2 : Find all authors who wrote a fiction book
authors_fiction = books.query("book(W, X, Y, Z), Y = 'fiction'")
print("\nAuthors who wrote a fiction book :")
print_query(authors_fiction, 'X')

# Your turn Query 3 : Find all books written before 1950
books_before_1950 = books.query("")
print("\nBooks written before 1950 :")
print_query(books_before_1950, 'X')

## Consult

To avoid using the `assertz()` function to create facts, we can use the `consult()` function to load a file containing the facts. This file must be a `.pl` file :

In [ ]:
family = Prolog()
family.consult("family.pl")

result = family.query("grandparent(X, Y)")
print(list(result))

## Lists in Prolog

Prolog as a built-in list structure which ressembles the formalism of languages such as Scheme or Lisp. The list is built upon two main blocks, the `Head` and the `Tail`, this allows us to do recursion in the list :

In [ ]:
sum = Prolog()
sum.consult("sum.pl")

result = sum.query("sum_list([1, 2, 3, 4, 5], Sum)")

print_query(result, 'Sum')

This code is defining a predicate `sum_list` in Prolog, which calculates the sum of a list of numbers.

The predicate takes two arguments: 
 - the first argument is the list of numbers
 - the second argument is the sum of those numbers.

The first line of the code `sum_list([], 0)` is a base case that defines the sum of an empty list as 0.

The second line of the code `sum_list([Head|Tail], Sum)` defines the recursive case, where `Head` is the first element of the list and `Tail` is the rest of the list.

The recursive case then calls the `sum_list` predicate again with the Tail argument to calculate the sum of the remaining elements of the list, and stores that sum in the `TailSum` variable.

Finally, the sum of Head and TailSum is computed using the `is` operator, and the result is stored in the `Sum` variable. This Sum value is returned as the final answer of the `sum_list` predicate.

Overall, the `sum_list` predicate recursively computes the sum of a list of numbers by breaking it down into smaller subproblems (i.e., the sum of the tail of the list), and then combining the results to get the final answer.

## Meta Programming

In [ ]:
meta = Prolog()
meta.consult("meta.pl")


result = meta.query("atoms_to_preds([foo, bar, baz], Preds).")
print(list(result))

This code defines a predicate `atoms_to_preds` that takes a list of atoms and returns a list of two-argument predicates with those names. Each predicate takes two arguments of any type and simply returns those arguments as a tuple.

To accomplish this, `atoms_to_preds` uses several meta-predicates. First, it constructs the names of the predicates using atom_concat and then converts those names to actual predicate terms using `atom_to_term`. It then combines these predicates into two-argument tuples using the Prolog tuple syntax.

Finally, the code uses the Prolog query system to demonstrate how atoms_to_preds can be used to create a list of predicates from a list of atoms. The query `atoms_to_preds([foo, bar, baz], Preds)` returns the list `[(foo_1, foo_2), (bar_1, bar_2), (baz_1, baz_2)]`, which demonstrates how meta-programming can be used to dynamically generate code.

## Real Database Example

Now that we have looked at the basics from Prolog, we can do some exercises on a real database. We will use the [student-loan](https://archive.ics.uci.edu/ml/datasets/Student+Loan+Relational) database from the UCI Machine Learning Repository. It is a practical database as facts are already stored in the `.pl` files we can use with Prolog.

This database contains informations about 1000 students, their loan payement status, employement status, and other facts. The document of the data is available in the `db/student-loan.names` file.

In [ ]:
#Load all the database files
import os

files = os.listdir("db/")

students_UCI = Prolog()

for file in files:
    if file.endswith(".pl"):
        # Uncomment the right line below to load the database
        
        #students_UCI.consult("db/" + file)
        #students_UCI.assertz("db/" + file)
        #students_UCI.query("db/" + file)

In [ ]:
# Let's find all the students who are enrolled in some institution
result = students_UCI.query("enrolled(X, _, _)")

print("All students who are enrolled :")
print_query(result, 'X')

In [ ]:
# Let's now find all the students who have filed for bankrupcy
result = students_UCI.query("filed_for_bankrupcy(X)")
print("All students who have filed for bankrupcy :")
print_query(result, 'X')

In [ ]:
# We can also combine queries in Prolog by using the comma operator as a logical AND
result = students_UCI.query("enrolled(X, _, _), filed_for_bankrupcy(X)")
print("All students who are enrolled and have filed for bankrupcy :")
print_query(result, 'X')

Now, you turn to play with the database and try to find all the students who are enlist in the `fire_department`, filed for bankrupcy AND who are at the `ucb` university.

In [ ]:
results = students_UCI.query("")
print("All students who are enrolled at UCB and are enlisted in the fire department :")
print_query(results, 'X')


## Conclusion
Prolog is a powerful and unique programming language that is particularly well-suited for problems related to artificial intelligence and natural language processing. Its declarative syntax and logical programming paradigm make it easy to learn and use, while its efficient backtracking search algorithm and built-in support for data types make it a powerful tool for searching through large databases and exploring multiple solutions. Although it has some limitations, Prolog has been successfully used in several real-world applications and is a valuable tool for any programmer interested in solving complex problems.

## References :

- [PyPi Documentation](https://pypi.org/project/pyswip/)
- [SWI-Prolog Documentation](https://www.swi-prolog.org/pldoc/doc_for?object=manual)
- [Cut & Negation Tutorial](https://www.cs.uleth.ca/~gaur/post/prolog-cut-negation/)
- [UCI Machine Learning Repository - Student Loan Dataset](https://archive.ics.uci.edu/ml/machine-learning-databases/student-loan/)